In [1]:
# -*- coding: utf-8 -*-  
import datetime
# import sys
# reload(sys)
# sys.setdefaultencoding('gbk') 
#过滤指定时间段的数据
```
简单数据处理,读取目标文件,然和筛选指定日期的数据,最后再和另一张表做连接,最后分组统计
```
begin_date=datetime.datetime.strptime("2017-09-07 08:00:00", '%Y-%m-%d %H:%M:%S')

delta=datetime.timedelta(days=1)
end_date=begin_date+delta
# begin_date="2017-09-20 08:00:00"
# end_date="2017-09-21 08:00:00"
print begin_date
print end_date
print "%s %s"%(begin_date,end_date)

2017-09-07 08:00:00
2017-09-08 08:00:00
2017-09-07 08:00:00 2017-09-08 08:00:00


In [2]:
# url ="jdbc:mysql://127.0.0.1:3306/jhgj?serverTimezone=UTC"  
# db_table = "pms_excel_commondata"  
# db_user="root"
# db_passwd="123456"
# def readTable(table):
#     # df = sqlContext.read.option("url", url)\
#     # .option("dbtable", table)\
#     # .option('user', "root")\
#     # .option('Parquet','jhgj')\
#     # .option('password', "123456")\
#     # .load()
#     return sqlContext.read.jdbc(url=url,table=table,properties={'user':db_user,'password':db_passwd,'driver':'com.mysql.jdbc.Driver'})

In [3]:
#读取出入库数据
warsDf = spark.read.csv("res/spark/2017-09-21_001_crkb.csv",encoding="utf-8",header=True,sep='\t')

In [4]:
#读取白名单数据
whiteDf=spark.read.csv("res/spark/PMS_CRTEXT_WHITE_LIST.csv",encoding="utf-8",header=True,sep=',',quote='"')
whiteDf.show(2)
whiteDf.printSchema()
# 读取字段映射
colMap=spark.read.csv("res/spark/PMS_EXCEL_COMMONDATA.csv",encoding="utf-8",header=True,sep=',',quote='"')
colMap.show(2)
colMap.printSchema()

+---+-----------+--------+
| id|SAP_BJ_CODE| CR_TEXT|
+---+-----------+--------+
|  1|60065710201|GD交货销售订单|
|  2|60065720201|GD交货销售订单|
+---+-----------+--------+
only showing top 2 rows

root
 |-- id: string (nullable = true)
 |-- SAP_BJ_CODE: string (nullable = true)
 |-- CR_TEXT: string (nullable = true)

+--------+--------+---------+-----------+--------+
|CELLNAME|CELLTYPE|CELLINDEX|   CELLCODE|EXCEL_ID|
+--------+--------+---------+-----------+--------+
| 系统包件订单号|  STRING|        7|BJ_ORDERNUM|       2|
|    计划数量|     INT|       11|    PLANNUM|       2|
+--------+--------+---------+-----------+--------+
only showing top 2 rows

root
 |-- CELLNAME: string (nullable = true)
 |-- CELLTYPE: string (nullable = true)
 |-- CELLINDEX: string (nullable = true)
 |-- CELLCODE: string (nullable = true)
 |-- EXCEL_ID: string (nullable = true)



In [5]:
cols=colMap.filter(colMap.EXCEL_ID==11).collect()
rows=cols
for row in rows:
    rowName=row["CELLNAME"]
    rowCode=row["CELLCODE"]
    print rowName,rowCode
    warsDf=warsDf.withColumnRenamed(rowName,rowCode)

库存地点描述 CKMARK
输入日期 CRDATE
移动类型文本 CRFLAG
数量 CRNUM
输入时间 CRTIME
供应商 FACTORY
抬头文本 HEAD_TEXT
物料 SAP_BJ_CODE
收货库存地点描述 SHCKMARK


In [6]:
from pyspark.sql.functions import *
warsDf=warsDf.withColumn("str_time",concat_ws(" ",warsDf.CRDATE,warsDf.CRTIME))
#日期转换,spark2.2可用
warsDf=warsDf.withColumn("date_time",to_timestamp("str_time","yyyy-MM-dd HH:mm:ss"))
warsDf.printSchema()

root
 |-- 物料凭证: string (nullable = true)
 |-- SAP_BJ_CODE: string (nullable = true)
 |-- 物料描述: string (nullable = true)
 |-- 移动类型: string (nullable = true)
 |-- CRFLAG: string (nullable = true)
 |-- CKMARK: string (nullable = true)
 |-- SHCKMARK: string (nullable = true)
 |-- CRTIME: string (nullable = true)
 |-- 工厂: string (nullable = true)
 |-- 单位文本: string (nullable = true)
 |-- HEAD_TEXT: string (nullable = true)
 |-- 资产: string (nullable = true)
 |-- 原因: string (nullable = true)
 |-- 订单: string (nullable = true)
 |-- 预留: string (nullable = true)
 |-- 名称 1: string (nullable = true)
 |-- 移动原因描述: string (nullable = true)
 |-- 客户: string (nullable = true)
 |-- 客户名称: string (nullable = true)
 |-- FACTORY: string (nullable = true)
 |-- 名称: string (nullable = true)
 |-- CRNUM: string (nullable = true)
 |-- CRDATE: string (nullable = true)
 |-- str_time: string (nullable = false)
 |-- date_time: timestamp (nullable = true)



In [7]:
warsDf=warsDf.orderBy(desc("date_time")).filter("date_time >='%s' and date_time<'%s'"%(begin_date,end_date) )


In [19]:
crtyp=spark.read.csv("res/spark/PMS_CR_TYPE_DECTIONAL.csv",encoding="utf-8",header=True,sep=',',quote='"')
crtyp.printSchema()
crtyp.show(3)

root
 |-- VALUEID: string (nullable = true)
 |-- VALUENAME: string (nullable = true)
 |-- TYPE_ID: string (nullable = true)
 |-- TYPENAME: string (nullable = true)
 |-- GOODFLAG: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)

+-------+----------+-------+--------+--------+----+----+----+
|VALUEID| VALUENAME|TYPE_ID|TYPENAME|GOODFLAG| _c5| _c6| _c7|
+-------+----------+-------+--------+--------+----+----+----+
|      1|有关销售订单库存收货|      4|      入汇|       1|null|null|null|
|      2|   寄售库存的收货|      1|      入汇|       1|null|null|null|
|      3|   采购订单的收货|      1|      入汇|       1|null|null|null|
+-------+----------+-------+--------+--------+----+----+----+
only showing top 3 rows



In [21]:
# df=readTable("pms_excel_commondata")
joinDf=warsDf.join(crtyp,crtyp.VALUENAME==warsDf.CRFLAG,"left")
joinDf.show(2)

+----------+-----------+--------------------+----+-------+---------+--------+--------+----+----+---------+----+----+----+----------+------+-----------+----+----+-------+----+------+----------+-------------------+-------------------+-------+---------+-------+--------+--------+----+----+----+
|      物料凭证|SAP_BJ_CODE|                物料描述|移动类型| CRFLAG|   CKMARK|SHCKMARK|  CRTIME|  工厂|单位文本|HEAD_TEXT|  资产|  原因|  订单|        预留|  名称 1|     移动原因描述|  客户|客户名称|FACTORY|  名称| CRNUM|    CRDATE|           str_time|          date_time|VALUEID|VALUENAME|TYPE_ID|TYPENAME|GOODFLAG| _c5| _c6| _c7|
+----------+-----------+--------------------+----+-------+---------+--------+--------+----+----+---------+----+----+----+----------+------+-----------+----+----+-------+----+------+----------+-------------------+-------------------+-------+---------+-------+--------+--------+----+----+----+
|5004707447|66031181701|60311H青少年书桌架B266B...| X03|成品库采购入库|物流工业园库存地点|    null|07:58:08|6810|   包|     null|null|6332|null|000

In [22]:
joinDf.printSchema()
df=joinDf.select("SAP_BJ_CODE","CRNUM","TYPENAME")
# theader.show()

root
 |-- 物料凭证: string (nullable = true)
 |-- SAP_BJ_CODE: string (nullable = true)
 |-- 物料描述: string (nullable = true)
 |-- 移动类型: string (nullable = true)
 |-- CRFLAG: string (nullable = true)
 |-- CKMARK: string (nullable = true)
 |-- SHCKMARK: string (nullable = true)
 |-- CRTIME: string (nullable = true)
 |-- 工厂: string (nullable = true)
 |-- 单位文本: string (nullable = true)
 |-- HEAD_TEXT: string (nullable = true)
 |-- 资产: string (nullable = true)
 |-- 原因: string (nullable = true)
 |-- 订单: string (nullable = true)
 |-- 预留: string (nullable = true)
 |-- 名称 1: string (nullable = true)
 |-- 移动原因描述: string (nullable = true)
 |-- 客户: string (nullable = true)
 |-- 客户名称: string (nullable = true)
 |-- FACTORY: string (nullable = true)
 |-- 名称: string (nullable = true)
 |-- CRNUM: string (nullable = true)
 |-- CRDATE: string (nullable = true)
 |-- str_time: string (nullable = false)
 |-- date_time: timestamp (nullable = true)
 |-- VALUEID: string (nullable = true)
 |-- VALUENAME: string (nul

In [12]:
crdate=begin_date.strftime("%Y-%m-%d")
crdate
df.registerTempTable("crData")
spark.sql("select t.*,'%s' as CRDATE from crData  t"%crdate).registerTempTable("crData1")

In [26]:
data1=spark.sql("select t.SAP_BJ_CODE,sum(t.CRNUM)as CRNUM,t.TYPENAME from crData1 t group by t.SAP_BJ_CODE,t.TYPENAME,t.CRDATE")
data1.describe()

DataFrame[summary: string, SAP_BJ_CODE: string, CRNUM: string, TYPENAME: string]

In [30]:
data1.where("TYPENAME IS NULL").count()
data1.count()

13130